In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
db_config = {
    'user': 'praktikum_student',
    'pwd': 'Sdf4$2;d-d30pp',
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432,
    'db': 'data-analyst-final-project-db'
}

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db']
)

engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [3]:
def query(engine):
    
    def execute(query):
        return pd.io.sql.read_sql(query, con = engine)
    
    return execute

In [4]:
execute_query = query(engine)

In [5]:
query = "SELECT COUNT(book_id) FROM books WHERE publication_date > '2000-01-01'"

execute_query(query)

,count
0,819


In [6]:
query = '''
    SELECT title, AVG(rating), COUNT(review_id) FROM books 
    INNER JOIN ratings ON books.book_id = ratings.book_id
    LEFT JOIN reviews ON books.book_id = reviews.book_id
    GROUP BY books.title
'''

execute_query(query)

,title,avg,count
0,The Count of Monte Cristo,4.217391,115
1,Count Zero (Sprawl #2),2.500000,4
2,The Botany of Desire: A Plant's-Eye View of th...,3.500000,4
3,The Poisonwood Bible,4.363636,110
4,The Canterbury Tales,3.333333,18
...,...,...,...
994,Of Love and Other Demons,4.500000,4
995,In the Heart of the Sea: The Tragedy of the Wh...,3.333333,9
996,Welcome to Temptation (Dempseys #1),5.000000,4
997,World's End (The Sandman #8),4.500000,4


In [7]:
query = '''
    SELECT publisher, COUNT(subquery.book_id) FROM publishers
    LEFT JOIN (
        SELECT book_id, publisher_id FROM books WHERE num_pages > 50
    ) AS subquery ON publishers.publisher_id = subquery.publisher_id
    GROUP BY publishers.publisher
    ORDER BY COUNT(subquery.book_id) DESC
    LIMIT 1
'''

execute_query(query)

,publisher,count
0,Penguin Books,42


In [9]:
query = '''
    SELECT author, AVG(avg) FROM books 
    RIGHT JOIN
    (SELECT book_id, COUNT(book_id), AVG(rating) 
    FROM ratings 
    GROUP BY book_id 
    HAVING COUNT(book_id) > 50) AS subquery
    ON books.book_id = subquery.book_id
    LEFT JOIN authors ON authors.author_id = books.author_id
    GROUP BY author
    ORDER BY avg DESC
'''

execute_query(query)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


In [19]:
query = '''
    SELECT AVG(count) FROM
    (SELECT reviews.username, COUNT(review_id) FROM reviews RIGHT JOIN 
    (SELECT username FROM ratings GROUP BY username HAVING COUNT(rating_id) > 50) AS subquery
    ON reviews.username = subquery.username
    GROUP BY reviews.username) AS subquery2
'''

execute_query(query)

,avg
0,24.333333
